In [ ]:
import pandas as pd
import os
import sys
import json

In [ ]:
if sys.platform.startswith("linux"):
    data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif_download_standalone/"
elif sys.platform == "darwin":
    data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
else:
    print("Not linux or mac!")

checklist_name = "uksi-moths-keys-nodup"

gbif_img_loc = os.path.join(data_dir,"gbif_images")

df = pd.read_csv(os.path.join("../species_checklists",checklist_name+".csv"))

In [ ]:
df["n_imgs"] = ""
for idx, row in df.iterrows():
    
    family  = row["family_name"]
    genus   = row["genus_name"]
    species = row["gbif_species_name"]
    
    # Check if directory exists 
    species_dir = (
        os.path.join(gbif_img_loc,family,genus,species)
        )
    
    if os.path.isdir(species_dir):
        n_images_on_disk = len(
            [f for f in os.listdir(species_dir) if f.lower().endswith('.jpg') ]
            )
        # print(f"{species} Count files method has", n_images_on_disk, "images")
        
        # Load metadata
        try:
            
            with open(os.path.join(species_dir,"meta_data.json")) as file:
            
                meta_data = json.load(file)
            
            # Count the number of images for this species
            md2_n_imgs_downloaded = 0

            try:
                # 2nd way of counting images
                md2 = pd.read_json(
                    os.path.join(species_dir,"meta_data.json"), orient = 'index'
                    )
                
                if md2.empty:
                    md2_n_imgs_downloaded = 0
                else:
                    md2_n_imgs_downloaded = md2["image_is_downloaded"].sum()

                # print(f"{species} Count dataframe metadata has", md2_n_imgs_downloaded, "images")            

            except Exception as e:
                print(e)
                print(f"{species} error counting dataframe way: {e}")
                        
            # Do n images match?
            if n_images_on_disk == md2_n_imgs_downloaded:
                # print(f"N images match for {species_dir}")
                pass
            else:
                print(
                    f"Mismatch! File count {n_images_on_disk}, "
                    f"metadata has {md2_n_imgs_downloaded}, {species_dir}"
                )
        except Exception as e:
            pass
            print(f"No metadata for {species_dir}")
            
    else:
        
        n_images_on_disk = 0
    
    # print(f"{species} has {n_images_on_disk} images")
    
    # Record this
    df.loc[idx, "n_imgs"] = n_images_on_disk
    
    

In [ ]:
df

In [ ]:
species_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/gbif_images/tests5/Sesiidae/Paranthrene/Paranthrene tabaniformis"

n_images_on_disk = len(
        [f for f in os.listdir(species_dir) if f.lower().endswith('.jpg') ]
        )
print(n_images_on_disk)

# 2nd way of counting images
md2 = pd.read_json(
        os.path.join(species_dir,"meta_data.json"), orient = 'index'
        )

if md2.empty:
        md2_n_imgs_downloaded = 0
else:
        md2_n_imgs_downloaded = md2["image_is_downloaded"].sum()
        
print(md2_n_imgs_downloaded)

In [ ]:
for f in os.listdir(species_dir):
    if f == "meta_data.json":
        continue
    if not md2.loc[f]["image_is_downloaded"]:
        print(f)

# "3905445525.jpg" in md2.index

In [ ]:
os.path.exists(os.path.join(species_dir,"3003600761.jpg"))

In [ ]:
media_df = pd.read_csv("/Users/lbokeria/Documents/projects/gbif-species-trainer-data/dwca_preprocessed/multimedia_lepidoptera_20231018.csv")


In [ ]:
a = media_df[media_df["coreid"] == 3003600761]

In [ ]:
image_url = a["identifier"].iloc[0]
image_url

In [ ]:
"https://files.plutof.ut.ee/public/orig/10/4D/104D9B04653A2839360BA8E4F16F5E56B2D8C401125459CFEEE33F848554724F.CR2"

In [ ]:
from urllib.request import urlretrieve

write_location = "/Users/lbokeria/Desktop"

# print("Trying   the image download", image_url)
image_write_path = os.path.join(write_location,str(124)+".jpg")
try:
    urlretrieve(image_url, image_write_path)
    print("success")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Save the df
# df.to_csv(os.path.join("../data_stats_files/","data_stats_"+checklist_name+".csv"))